In [2]:
]activate .

  Activating new environment at `~/ibmgithub/jlale/src/Project.toml`


In [3]:
using PyCall
using Pandas
sk       = pyimport("sklearn")
np       = pyimport("numpy")
pd       = pyimport("pandas")
dt       = pyimport("lale.datasets")
pickle   = pyimport("pickle")
lale     = pyimport("lale")
RF       = pyimport("sklearn.ensemble").RandomForestRegressor
LinReg   = pyimport("sklearn.linear_model").LinearRegression
PCA      = pyimport("sklearn.decomposition").PCA
ICA      = pyimport("sklearn.decomposition").FastICA
Tree     = pyimport("sklearn.tree").DecisionTreeRegressor
Pipeline = pyimport("sklearn.pipeline").Pipeline

PyObject <class 'sklearn.pipeline.Pipeline'>

In [4]:
# @pyimport lale.lib.lale as lb
lb             = pyimport("lale.lib.lale")
Hyperopt       = lb.Hyperopt
GridSearchCV   = lb.GridSearchCV
NoOp           = lb.NoOp
ConcatFeatures = lb.ConcatFeatures

# @pyimport lale.operators as lops
lops          = pyimport("lale.operators")
make_pipeline = lops.make_pipeline
make_choice   = lops.make_choice
make_union    = lops.make_union

PyObject <function make_union at 0x7f963161a440>

In [5]:
# define lale operators
import Base: >>, |, +
>>(a::PyObject,b::PyObject) = make_pipeline(a,b)
+(a::PyObject,b::PyObject)  = make_union(a,b)  # UTF \oplus
|(a::PyObject,b::PyObject)  = make_choice(a,b) # UTF \times

| (generic function with 33 methods)

In [6]:
# load data
(train_X, train_y), (test_X, test_y) = dt.california_housing_df()
pd.concat([train_X.head(), train_y.head()], axis=1)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,3.2596,33.0,5.017657,1.006421,2300.0,3.691814,32.71,-117.03,1.030
1,3.8125,49.0,4.473545,1.041005,1314.0,1.738095,33.77,-118.16,3.821
2,4.1563,4.0,5.645833,0.985119,915.0,2.723214,34.66,-120.48,1.726
3,1.9425,36.0,4.002817,1.033803,1418.0,3.994366,32.69,-117.11,0.934
4,3.5542,43.0,6.268421,1.134211,874.0,2.300000,36.78,-119.80,0.965


In [7]:
# pipeline 1
pca_tree_planned = Pipeline(steps=[("tfm", PCA()), ("estim", Tree())])
pca_tree_planned.fit(train_X, train_y)
predicted = pca_tree_planned.predict(test_X);
print("R2 score: ",sk.metrics.r2_score(test_y, predicted))

R2 score: 0.48907967672966424

In [8]:
# pipeline 2
pipeline    = PCA() >> Tree()
trainedhopt = pipeline.auto_configure(train_X, train_y, optimizer=Hyperopt,      cv=3, max_evals=10, verbose=true)
predicted   = trainedhopt.predict(test_X);
print("R2 score: ",sk.metrics.r2_score(test_y, predicted))

100%|███████| 10/10 [00:05<00:00,  1.74trial/s, best loss: -0.5164011725861685]R2 score: 0.4960088670497377

In [16]:
# pipeline 2
pipeline    = PCA() >> Tree()
trainedhopt = pipeline.auto_configure(train_X, train_y, optimizer=Hyperopt,      cv=3, max_evals=10, verbose=true)
predicted   = trainedhopt.predict(test_X);
print("R2 score: ",sk.metrics.r2_score(test_y, predicted))


100%|███████| 10/10 [00:06<00:00,  1.55trial/s, best loss: -0.5144186202969583]R2 score: 0.4803667195126011

In [17]:
# pipeline 3
planned    = (PCA() + NoOp())   >>  (LinReg() | Tree())
trained    = planned.auto_configure(train_X, train_y, optimizer=Hyperopt, cv=3,  max_evals=3, verbose=true)
predicted  = trained.predict(test_X);
print("R2 score: ",sk.metrics.r2_score(test_y, predicted))


100%|█████████| 3/3 [00:01<00:00,  1.71trial/s, best loss: -0.6110613494521347]R2 score: 0.575751075388504

In [18]:
# pipeline 5
pipeline    = ((PCA(svd_solver="auto") + ICA()) ) >> Tree()
trainedgrid = pipeline.auto_configure(train_X, train_y, optimizer=Hyperopt,      max_evals=5, cv=3,verbose=true)
predicted   = trainedgrid.predict(test_X);
print("R2 score: ",sk.metrics.r2_score(test_y, predicted))


100%|█████████| 5/5 [00:05<00:00,  1.07s/trial, best loss: -0.5813302250875374]R2 score: 0.5625205708723776